In [1]:
# use python 3.6 to download option data from nsepy
import time
import datetime
from datetime import date

import multiprocessing as mp
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dateutil.relativedelta import relativedelta
# from pandas.tseries.offsets import DateOffset
# from pandas.tseries.offsets import MonthOffset

# finnacial data
# from nsepy import get_history
# from nsepy.derivatives import get_expiry_date
import mibian

# import iv calculation script for parallel processing
import iv_calc

In [2]:
# read in OHLC data
option_raw = pd.read_csv('../data_output/nifty_15yr_data.csv', parse_dates = ['Date', 'Expiry'], infer_datetime_format = True, header = 0)
option_raw.info
option_raw.head(n=3)

,Date,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
0,2006-04-07,NIFTY,2006-06-29,PE,3600.0,0.0,0.0,0.0,143.6,0.0,180.15,0,0.0,NaN,0,0,3454.80
1,2006-04-10,NIFTY,2006-06-29,PE,3600.0,0.0,0.0,0.0,143.6,0.0,163.55,0,0.0,NaN,0,0,3478.45
2,2006-04-12,NIFTY,2006-06-29,PE,3600.0,0.0,0.0,0.0,143.6,0.0,245.15,0,0.0,NaN,0,0,3380.00


In [3]:
# subset relevant columns; chekc date
option = option_raw[['Date', 'Expiry', 'Option Type', 'Strike Price', 'Close', 'Underlying']]
print(option.dtypes)
print(option.shape)
option.head(3)

Date            datetime64[ns]
Expiry          datetime64[ns]
Option Type             object
Strike Price           float64
Close                  float64
Underlying             float64
dtype: object
(155846, 6)


,Date,Expiry,Option Type,Strike Price,Close,Underlying
0,2006-04-07,2006-06-29,PE,3600.0,143.6,3454.80
1,2006-04-10,2006-06-29,PE,3600.0,143.6,3478.45
2,2006-04-12,2006-06-29,PE,3600.0,143.6,3380.00


In [4]:
option['Option Type'].value_counts() # should be equal

CE    77948
PE    77898
Name: Option Type, dtype: int64

In [5]:
future_raw = pd.read_csv('../data_output/nifty_futures_15yr.csv', parse_dates = ['Date', 'Expiry'], infer_datetime_format = True, header = 0)
future_raw.tail(n=3)

,Date,Change in OI,Close,Expiry,High,Last,Low,Number of Contracts,Open,Open Interest,Settle Price,Symbol,Turnover,Underlying,Volume,expiry
11586,2020-12-29,NaN,13932.60,NaT,13967.60,NaN,13859.90,NaN,13910.35,NaN,NaN,NaN,2.515423e+11,NaN,439593961.0,2020-12-31
11587,2020-12-30,NaN,13981.95,NaT,13997.00,NaN,13864.95,NaN,13980.90,NaN,NaN,NaN,2.463312e+11,NaN,380681073.0,2020-12-31
11588,2020-12-31,NaN,13981.75,NaT,14024.85,NaN,13936.45,NaN,13970.00,NaN,NaN,NaN,2.630582e+11,NaN,452410109.0,2020-12-31


In [6]:
# move relevant columns to a new dataframe
future = future_raw[['Date', 'expiry', 'Close']]
print(future.dtypes)
print(future.shape)
future.head(3)

Date      datetime64[ns]
expiry            object
Close            float64
dtype: object
(11589, 3)


,Date,expiry,Close
0,2004-11-01,2005-01-27,1797.75
1,2004-11-02,2005-01-27,1813.70
2,2004-11-03,2005-01-27,1837.40


In [7]:
future['expiry'] = pd.to_datetime(future['expiry'])
print(future.dtypes)
print(future.shape)
future.head(3)

Date      datetime64[ns]
expiry    datetime64[ns]
Close            float64
dtype: object
(11589, 3)


C:\Users\gurdeep.singh\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,expiry,Close
0,2004-11-01,2005-01-27,1797.75
1,2004-11-02,2005-01-27,1813.70
2,2004-11-03,2005-01-27,1837.40


In [8]:
# load lending rates - this may not be needed
lending = pd.read_csv('../data_output/india_lending_rate.csv')
print(lending.shape)
lending.head(3)

(1, 47)


,Country Name,Country Code,Indicator Name,Indicator Code,1978,1979,1980,1981,1982,1983,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,India,IND,Lending interest rate (%),FR.INR.LEND,13.5,14.5,16.5,16.5,16.5,16.5,...,10.166667,10.604167,10.291667,10.25,10.008333,9.6725,9.508333,9.454167,9.46625,9.15


### Join Future price with Option 

**Note on Futures prices**

* We propose to use futures price for option pricing in place of using underlying and interest rates.
* This is due to prices being affected  because dividends and other corporate actions - bonus, rights issue etc.
* Before we join the 2 dataframes we need to ensure that combination of data and expiry is unique. 
* This should have been the case but better to verify.

In [9]:
future['Date'].nunique()

4012

In [10]:
future['expiry'].nunique()

191

In [11]:
# combine both columns and check nrows
x = future['Date'].astype(str) + future['expiry'].astype(str)
print(x.shape)
x.nunique() # should be similar to number of rows in futures dataframe

(11589,)


11589

In [12]:
# similar check for option before joining
y = option['Date'].astype(str)+option['Expiry'].astype(str)
print(y.shape)
y.nunique() # we have less rows in options than futures

(155846,)


9160

In [13]:
option2 = option.copy() #  create a copy just in case
option2 = pd.merge(option, future[['Date','expiry', 'Close']], left_on = ['Date', 'Expiry'], right_on = ['Date', 'expiry'], how = 'left')

print(option2.shape)
option2.head()

(155846, 8)


,Date,Expiry,Option Type,Strike Price,Close_x,Underlying,expiry,Close_y
0,2006-04-07,2006-06-29,PE,3600.0,143.6,3454.80,2006-06-29,3454.80
1,2006-04-10,2006-06-29,PE,3600.0,143.6,3478.45,2006-06-29,3478.45
2,2006-04-12,2006-06-29,PE,3600.0,143.6,3380.00,2006-06-29,3380.00
3,2006-04-13,2006-06-29,PE,3600.0,143.6,3345.50,2006-06-29,3345.50
4,2006-04-17,2006-06-29,PE,3600.0,143.6,3425.15,2006-06-29,3425.15


In [14]:
# check if "Expiry" and "expiry" are equal; trust but verify
option2['Expiry'].equals(option2.expiry)

# without pandas 
# np.where(option2['Expiry'] != option2['expiry'],1,0).sum()

True

In [15]:
# drop and rename columns 

option2.drop('Expiry', axis = 1, inplace = True)
option2.rename(columns = {'Close_y' : 'futures', 'Close_x' : 'close'}, inplace = True)

# add column for ATM strike price and days to expiry
option2['days2expiry'] = (option2.expiry - option2.Date).dt.days

atm_strike = round(option2.loc[0,'Underlying']/100)*100
option2['atm_strike'] = atm_strike
print(option2.shape)
option2.tail(3)

(155846, 9)


,Date,Option Type,Strike Price,close,Underlying,expiry,futures,days2expiry,atm_strike
155843,2020-12-29,CE,14900.0,0.8,NaN,2020-12-31,13932.60,2,3500.0
155844,2020-12-30,CE,14900.0,0.7,13981.95,2020-12-31,13981.95,1,3500.0
155845,2020-12-31,CE,14900.0,0.1,13981.75,2020-12-31,13981.75,0,3500.0


### Option pricing

https://github.com/OptionsnPython/Option-strategies-backtesting-in-Python/blob/main/Option%20Greeks%20Strategies%20%26%20Backtesting%20in%20Python.%20File%202%20-Greeks%20in%20Python%20using%20mibian.ipynb

#### Option pricing depends on

* Underlying price - Nifty in this case 

In [16]:
# calculate IV for single option; create test dataframe
test_put = option2.loc[:300,:]
print(test_put.shape)
print(test_put.dtypes)
print(test_put['Option Type'].value_counts())
test_put.head(3)

(301, 9)
Date            datetime64[ns]
Option Type             object
Strike Price           float64
close                  float64
Underlying             float64
expiry          datetime64[ns]
futures                float64
days2expiry              int64
atm_strike             float64
dtype: object
PE    161
CE    140
Name: Option Type, dtype: int64


,Date,Option Type,Strike Price,close,Underlying,expiry,futures,days2expiry,atm_strike
0,2006-04-07,PE,3600.0,143.6,3454.80,2006-06-29,3454.80,83,3500.0
1,2006-04-10,PE,3600.0,143.6,3478.45,2006-06-29,3478.45,80,3500.0
2,2006-04-12,PE,3600.0,143.6,3380.00,2006-06-29,3380.00,78,3500.0


In [17]:
# create test df for call options
test_call = option2[option2['Option Type'] == 'CE'].head(300)
print(test_call.shape)
print(test_call.dtypes)
print(test_call['Option Type'].value_counts())
test_call.head(3)

(300, 9)
Date            datetime64[ns]
Option Type             object
Strike Price           float64
close                  float64
Underlying             float64
expiry          datetime64[ns]
futures                float64
days2expiry              int64
atm_strike             float64
dtype: object
CE    300
Name: Option Type, dtype: int64


,Date,Option Type,Strike Price,close,Underlying,expiry,futures,days2expiry,atm_strike
59,2006-04-07,CE,3600.0,83.0,3454.80,2006-06-29,3454.80,83,3500.0
60,2006-04-10,CE,3600.0,75.0,3478.45,2006-06-29,3478.45,80,3500.0
61,2006-04-12,CE,3600.0,45.0,3380.00,2006-06-29,3380.00,78,3500.0


In [18]:
# test mibian - replicate githb code for put option
# https://github.com/yassinemaaroufi/MibianLib

# underlying, strike, interest, days to expiry, volatility
put_price = mibian.BS([1.4565, 1.45, 1,30], volatility = 20).putPrice
print(put_price)

# iv - underlying, strike, interest rate, days to expiry
iv = mibian.BS([1.4565, 1.45, 1, 30], putPrice = put_price).impliedVolatility
print(iv)

0.029520257209636247
20.000457763671875


In [19]:
# test mibian - on our data using 5% interest rate asmp
# see below results are pretty close to actual values
# iv - underlying, strike, interest rate, days to expiry
iv = mibian.BS([3454.8, 3500, 0.05*365/83, 83], putPrice = 143.6).impliedVolatility
print(iv)

# underlying, strike, interest, days to expiry, volatility
put_price = mibian.BS([3454.8, 3500, 0.05*365/83, 83], volatility = iv).putPrice
print(put_price)

18.24951171875
143.62637375839972


In [20]:
# test mibian - on our data using **futures price**
# see below results are pretty close to actual values
# iv - underlying, strike, interest rate, days to expiry
iv = mibian.BS([3458.15, 3500, 0, 83], putPrice = 143.6).impliedVolatility
print(iv)

# repeat put price calculation using futures price
put_price2 = mibian.BS([3458.15, 3500, 0, 83], volatility = iv).putPrice
print(put_price2)

18.37158203125
143.62370884999427


In [21]:
def put_iv(df):
    # lets write a function to calculate IV for put options
    try: 
        # iv - underlying, strike, interest rate, days to expiry
        return mibian.BS([df['futures'], df['Strike Price'], 0, df['days2expiry']], putPrice = df['close']).impliedVolatility
    except:
        return np.nan

In [22]:
# test the function on sample; apply to larger if it works
start = time.perf_counter()
test_put['put_iv'] = test_put.apply(put_iv, axis = 1)
end = time.perf_counter()

total = end-start
print(total)
test_put.tail()

690.4676882


C:\Users\gurdeep.singh\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Date,Option Type,Strike Price,close,Underlying,expiry,futures,days2expiry,atm_strike,put_iv
296,2006-05-24,PE,3600.0,201.0,3115.55,2006-07-27,3115.55,64,3500.0,0.000010
297,2006-05-25,PE,3600.0,625.0,3177.70,2006-07-27,3177.70,63,3500.0,67.382812
298,2006-05-26,PE,3600.0,625.0,3209.60,2006-07-27,3209.60,62,3500.0,71.762085
299,2006-05-29,PE,3600.0,630.0,3214.90,2006-07-27,3214.90,59,3500.0,75.195312
300,2006-05-30,PE,3600.0,499.8,3185.30,2006-07-27,3185.30,58,3500.0,44.021606


In [23]:
        # underlying, strike, interest rate, days2expiry
call_price = mibian.BS([1.4565, 1.45, 1, 30], volatility = 20).callPrice
print(call_price)

call_iv = mibian.BS([1.4565, 1.45, 1, 30], callPrice = 0.0359).impliedVolatility
print(call_iv)

0.03721154839277063
19.22607421875


In [24]:
def call_iv(df):
    # function to calculate IV of call option
    try:
        # underlying, strike, interest, days to expiry, volatility
        return mibian.BS([df['futures'], df['Strike Price'], 0, df['days2expiry']], callPrice = df['close']).impliedVolatility
    except:
        return np.nan

In [25]:
# test the function on sample; apply to larger if it works
start = time.perf_counter()
test_call['call_iv'] = test_call.apply(call_iv, axis = 1)
end = time.perf_counter()

total = end-start
print(total)

test_call.tail()

2.1475882999999385


,Date,Option Type,Strike Price,close,Underlying,expiry,futures,days2expiry,atm_strike,call_iv
620,2006-06-19,CE,3800.0,12.0,2916.90,2006-07-27,2916.90,38,3500.0,50.598145
621,2006-06-20,CE,3800.0,12.0,2861.30,2006-07-27,2861.30,37,3500.0,54.321289
622,2006-06-21,CE,3800.0,12.0,2923.45,2006-07-27,2923.45,36,3500.0,51.635742
623,2006-06-22,CE,3800.0,9.0,2994.75,2006-07-27,2994.75,35,3500.0,45.654297
624,2006-06-23,CE,3800.0,10.0,3042.70,2006-07-27,3042.70,34,3500.0,44.677734


### Calculate and explore IV for entire dataset

In [26]:
# def iv_calc(df):
#     # function to call put or call function
#     if (df['Option Type'] == 'PE'):
#         return df.apply
#     else:
#         return call_iv

In [27]:
test_iv = pd.concat([test_call, test_put])
test_iv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601 entries, 59 to 300
Data columns (total 11 columns):
Date            601 non-null datetime64[ns]
Option Type     601 non-null object
Strike Price    601 non-null float64
Underlying      601 non-null float64
atm_strike      601 non-null float64
call_iv         295 non-null float64
close           601 non-null float64
days2expiry     601 non-null int64
expiry          601 non-null datetime64[ns]
futures         601 non-null float64
put_iv          295 non-null float64
dtypes: datetime64[ns](2), float64(7), int64(1), object(1)
memory usage: 56.3+ KB


In [28]:
# with serial processing
start = time.perf_counter()
test_iv['option_iv'] = np.where(test_iv['Option Type'] == 'CE', test_iv.apply(call_iv, axis = 1), test_iv.apply(put_iv, axis = 1))
end = time.perf_counter()

total = end-start
print(total)

test_iv.head()

2155.9352718


,Date,Option Type,Strike Price,Underlying,atm_strike,call_iv,close,days2expiry,expiry,futures,put_iv,option_iv
59,2006-04-07,CE,3600.0,3454.80,3500.0,21.484375,83.00,83,2006-06-29,3454.80,NaN,21.484375
60,2006-04-10,CE,3600.0,3478.45,3500.0,19.165039,75.00,80,2006-06-29,3478.45,NaN,19.165039
61,2006-04-12,CE,3600.0,3380.00,3500.0,19.515991,45.00,78,2006-06-29,3380.00,NaN,19.515991
62,2006-04-13,CE,3600.0,3345.50,3500.0,24.904251,62.55,77,2006-06-29,3345.50,NaN,24.904251
63,2006-04-17,CE,3600.0,3425.15,3500.0,21.499634,64.90,73,2006-06-29,3425.15,NaN,21.499634


In [37]:
# get cpu count for parallelization
num_proc = mp.cpu_count()-1
print('Number of processors: ', num_proc)

Number of processors:  7


In [30]:
def parallelize_df(df, func, n_cores = num_proc):
    df_split = np.array_split(df, n_cores)
    pool = mp.Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [31]:
start = time.perf_counter()
test_iv = parallelize_df(test_iv, iv_calc.iv_function)
end = time.perf_counter()
end-start

800.9007739999997

In [38]:
option2.head()

,Date,Option Type,Strike Price,close,Underlying,expiry,futures,days2expiry,atm_strike
0,2006-04-07,PE,3600.0,143.6,3454.80,2006-06-29,3454.80,83,3500.0
1,2006-04-10,PE,3600.0,143.6,3478.45,2006-06-29,3478.45,80,3500.0
2,2006-04-12,PE,3600.0,143.6,3380.00,2006-06-29,3380.00,78,3500.0
3,2006-04-13,PE,3600.0,143.6,3345.50,2006-06-29,3345.50,77,3500.0
4,2006-04-17,PE,3600.0,143.6,3425.15,2006-06-29,3425.15,73,3500.0


In [ ]:
start = time.perf_counter()
option2 = parallelize_df(option2, iv_calc.iv_function)
end = time.perf_counter()
end-start

In [32]:
# plot call and put IV on a single chart
# 2 charts - put and call; 90, 45, 30, 15, 5 days out in each chart over all the years

In [33]:
# IV changes for a volatile period +/- 10 days for 2008, 2017 and 2020 crises
# get exact dates from previous blogpost

### Does the volatility smile exist?

In [34]:
# plot for option price 30 days to expiry with 4 months times 4 years
# same expiry date and same days2expiry

### Exploring Convexity

In [35]:
# find periods with > 3SD movement

In [36]:
# Plot change in premium of option that is 2SD OTM - both put and call